In [18]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import pickle

In [3]:
with open('example.htm', 'r') as file:
    soup = BeautifulSoup(file.read(),'lxml')

In [54]:
def gen_database(filename: str,season: str) -> dict:
    ''' 
    Creates a dictionary of units with their statistics.

    :param filename: str filepath to read from.
    :param save_filename: str filepath to write to.
    :param sem: adds the semester and year in which the unit was done. e.g 2020_S1
    :output: dictionary.

    '''
    
    with open(filename, "r") as f:
        contents = f.read()
        soup = BeautifulSoup(contents, "lxml")

    database = {}

    for article in soup.find_all("article"):
        # invited number first
        base = article.find(
            "div", attrs={"class": "CrossCategoryBlockRow TableContainer"}).find("tbody")
        
        invited = int(
            base
            .find("tr", attrs={'class': 'CondensedTabularOddRows'})
            .find('td')
            .text)
        
        responded = int(
            base
            .find("tr", attrs={'class': 'CondensedTabularEvenRows'})
            .find('td')
            .text)

        if responded <= 1: continue

        entry = {}
        entry["Responses"] = responded
        entry["Invited"] = invited
        entry["Season"] = season
        entry['Response Rate'] = responded/invited*100

        # Full unit code
        code = article.find("table").find_all("tr")[3].text

        # Filter out MALAYSIA, COMPOSITE, ALFRED, SAFRICA
        if any(location in code for location in ["MALAYSIA","ALFRED","SAFRICA","FLEXIBLE"]):
            continue

        entry["code"] = code
        entry["unit_code"] = code.split("_")[0][1:]
        # Do not display on datatable, used only for queries
        try:
            entry["Level"] = int(entry["unit_code"][3]) 
        except ValueError: 
            entry["Level"] = 0
        scores = []
        # Response categories, retrieve all tables
        for item_num,divs in enumerate(
            article.find_all("div", attrs={"class": "FrequencyBlock_HalfMain"})):

            score_table = divs.find_all("table")[1].tbody.find_all("tr") # Split by stats and chart

            # Extract the means and medians from their td element
            mean, median = list(
                map(lambda x: x.find("td").text,
                    score_table
                )
            )[1:3]

            # Attempt conversion, not sure if this activates...?
            try:
                mean, median = float(mean), float(median)
                entry[f'I{item_num+1}'] = [mean,median]
                scores.append([mean,median])
            except ValueError:
                print(f"score could not be converted: {code}, {mean}, {median}")

        entry['agg_score'] = [sum(map(
            lambda item:item[measure],scores))/len(scores)
            for measure in range(2)
            ]

        database[code] = entry
        # Serialize after each point
        '''
        with open(save_filename, "wb") as f:
            pickle.dump(database, f, pickle.HIGHEST_PROTOCOL)
        '''

    return pd.DataFrame(database).T

In [55]:
thing = gen_database('batch_conversion.htm','2020_S1')


In [56]:
thing

,Responses,Invited,Season,Response Rate,code,unit_code,Level,I1,I2,I3,...,I5,I6,I7,I8,I9,I10,I11,I12,I13,agg_score
ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01,41,115,2020_S1,35.652174,ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01,ACB1020,1,"[4.12, 4.18]","[4.34, 4.33]","[3.98, 4.0]",...,"[3.85, 3.92]","[3.75, 3.87]","[4.2, 4.18]","[3.8, 3.97]","[4.03, 4.03]","[4.05, 4.05]","[4.0, 4.09]","[4.22, 4.25]","[3.85, 3.89]","[4.001538461538462, 4.053076923076923]"
ACB1120_PENINSULA_ON-CAMPUS_ON_S1-01,13,65,2020_S1,20.0,ACB1120_PENINSULA_ON-CAMPUS_ON_S1-01,ACB1120,1,"[3.92, 4.0]","[4.08, 4.08]","[3.92, 3.94]",...,"[3.92, 4.0]","[3.85, 3.89]","[4.0, 4.0]","[3.77, 3.86]","[4.08, 4.06]","[4.0, 4.0]","[4.08, 4.06]","[3.92, 3.94]","[4.0, 4.0]","[3.946923076923077, 3.9676923076923076]"
ACB2120_PENINSULA_ON-CAMPUS_ON_S1-01,7,22,2020_S1,31.818182,ACB2120_PENINSULA_ON-CAMPUS_ON_S1-01,ACB2120,2,"[4.0, 4.63]","[4.29, 4.63]","[3.86, 4.25]",...,"[3.86, 4.0]","[3.71, 4.63]","[4.29, 4.33]","[3.57, 4.0]","[3.67, 4.5]","[4.14, 4.63]","[4.0, 4.63]","[3.86, 4.63]","[3.86, 4.63]","[3.9315384615384614, 4.470769230769232]"
ACB2220_PENINSULA_ON-CAMPUS_ON_S1-01,6,10,2020_S1,60.0,ACB2220_PENINSULA_ON-CAMPUS_ON_S1-01,ACB2220,2,"[4.5, 4.5]","[4.33, 4.5]","[4.33, 4.5]",...,"[4.0, 4.5]","[4.33, 4.5]","[4.5, 4.5]","[4.33, 4.5]","[4.25, 4.5]","[4.67, 4.75]","[4.2, 4.25]","[4.5, 4.5]","[4.33, 4.5]","[4.341538461538462, 4.5]"
ACC1100_CLAYTON_ON-CAMPUS_ON_S1-01,145,506,2020_S1,28.656126,ACC1100_CLAYTON_ON-CAMPUS_ON_S1-01,ACC1100,1,"[4.22, 4.27]","[4.1, 4.17]","[4.19, 4.26]",...,"[4.06, 4.12]","[4.19, 4.25]","[4.17, 4.29]","[4.06, 4.14]","[4.19, 4.22]","[4.35, 4.39]","[4.13, 4.21]","[4.03, 4.16]","[3.89, 3.97]","[4.111538461538462, 4.185384615384615]"


In [19]:
with open('test.pkl','wb') as file:
    pickle.dump(thing,file,pickle.HIGHEST_PROTOCOL)

In [20]:
with open('test.pkl','rb') as file:
    db = pickle.load(file)

In [45]:
thing[[f'I{n}' for n in range(1,14)]] =thing[[f'I{n}' for n in range(1,14)]].apply(lambda x:x[1])

In [49]:
for item in range(1,14): thing[f'I{item}'] = thing[f'I{item}'].apply(lambda x: x[0])

In [50]:
thing

,Responses,Invited,Season,Response Rate,code,unit_code,Level,I1,I2,I3,...,I5,I6,I7,I8,I9,I10,I11,I12,I13,agg_score
ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01,41,115,2020_S1,35.652174,ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01,ACB1020,0,4.12,4.34,3.98,...,3.85,3.75,4.20,3.80,4.03,4.05,4.00,4.22,3.85,"[4.001538461538462, 4.053076923076923]"
ACB1120_PENINSULA_ON-CAMPUS_ON_S1-01,13,65,2020_S1,20.0,ACB1120_PENINSULA_ON-CAMPUS_ON_S1-01,ACB1120,1,3.92,4.08,3.92,...,3.92,3.85,4.00,3.77,4.08,4.00,4.08,3.92,4.00,"[3.946923076923077, 3.9676923076923076]"
ACB2120_PENINSULA_ON-CAMPUS_ON_S1-01,7,22,2020_S1,31.818182,ACB2120_PENINSULA_ON-CAMPUS_ON_S1-01,ACB2120,1,4.00,4.29,3.86,...,3.86,3.71,4.29,3.57,3.67,4.14,4.00,3.86,3.86,"[3.9315384615384614, 4.470769230769232]"
ACB2220_PENINSULA_ON-CAMPUS_ON_S1-01,6,10,2020_S1,60.0,ACB2220_PENINSULA_ON-CAMPUS_ON_S1-01,ACB2220,2,4.50,4.33,4.33,...,4.00,4.33,4.50,4.33,4.25,4.67,4.20,4.50,4.33,"[4.341538461538462, 4.5]"
ACC1100_CLAYTON_ON-CAMPUS_ON_S1-01,145,506,2020_S1,28.656126,ACC1100_CLAYTON_ON-CAMPUS_ON_S1-01,ACC1100,1,4.22,4.10,4.19,...,4.06,4.19,4.17,4.06,4.19,4.35,4.13,4.03,3.89,"[4.111538461538462, 4.185384615384615]"


In [23]:
for key in thing:
    for i in range(9):
        thing[key][f'I{i}'] = thing[key]['scores'][i]

KeyError: 'scores'

In [62]:
e.loc[e.Responses>20][[f'I{n}' for n in range(9)]].apply(lambda x: x[-1])

,I0,I1,I2,I3,I4,I5,I6,I7,I8
ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01,4.22,4.10,4.19,3.87,4.06,4.19,4.17,4.06,4.19
ACC1100_CLAYTON_ON-CAMPUS_ON_S1-01,4.27,4.17,4.26,3.96,4.12,4.25,4.29,4.14,4.22


In [63]:
e['agg_score'] = [sum(map(lambda x:x[a], e[[f'I']]  ))]

,Responses,Invited,Semester,code,unit_code,Level,I0,I1,I2,I3,I4,I5,I6,I7,I8
ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01,41,115,S1,ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01,ACB1020,1,"[4.12, 4.18]","[4.34, 4.33]","[3.98, 4.0]","[3.83, 3.93]","[3.85, 3.92]","[3.75, 3.87]","[4.2, 4.18]","[3.8, 3.97]","[4.03, 4.03]"
ACB1120_PENINSULA_ON-CAMPUS_ON_S1-01,13,65,S1,ACB1120_PENINSULA_ON-CAMPUS_ON_S1-01,ACB1120,1,"[3.92, 4.0]","[4.08, 4.08]","[3.92, 3.94]","[3.77, 3.75]","[3.92, 4.0]","[3.85, 3.89]","[4.0, 4.0]","[3.77, 3.86]","[4.08, 4.06]"
ACB2120_PENINSULA_ON-CAMPUS_ON_S1-01,7,22,S1,ACB2120_PENINSULA_ON-CAMPUS_ON_S1-01,ACB2120,2,"[4.0, 4.63]","[4.29, 4.63]","[3.86, 4.25]","[4.0, 4.63]","[3.86, 4.0]","[3.71, 4.63]","[4.29, 4.33]","[3.57, 4.0]","[3.67, 4.5]"
ACB2220_PENINSULA_ON-CAMPUS_ON_S1-01,6,10,S1,ACB2220_PENINSULA_ON-CAMPUS_ON_S1-01,ACB2220,2,"[4.5, 4.5]","[4.33, 4.5]","[4.33, 4.5]","[4.17, 4.5]","[4.0, 4.5]","[4.33, 4.5]","[4.5, 4.5]","[4.33, 4.5]","[4.25, 4.5]"
ACC1100_CLAYTON_ON-CAMPUS_ON_S1-01,145,506,S1,ACC1100_CLAYTON_ON-CAMPUS_ON_S1-01,ACC1100,1,"[4.22, 4.27]","[4.1, 4.17]","[4.19, 4.26]","[3.87, 3.96]","[4.06, 4.12]","[4.19, 4.25]","[4.17, 4.29]","[4.06, 4.14]","[4.19, 4.22]"


In [15]:
e['scores'].apply(lambda x:[sum(a[i] for a in x)/len(x) for i in range(2)])

 ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01      [4.001538461538462, 4.053076923076923]
 ACB1120_PENINSULA_ON-CAMPUS_ON_S1-01     [3.946923076923077, 3.9676923076923076]
 ACB2120_PENINSULA_ON-CAMPUS_ON_S1-01     [3.9315384615384614, 4.470769230769232]
 ACB2220_PENINSULA_ON-CAMPUS_ON_S1-01                    [4.341538461538462, 4.5]
 ACC1100_CLAYTON_ON-CAMPUS_ON_S1-01        [4.111538461538462, 4.185384615384615]
Name: scores, dtype: object

In [39]:
def to_dataframe(db:dict) -> pd.DataFrame:
    '''
    Converts the input dict to a dataframe. 

    :param db: Dictionary of unit statistics.
    :output: Dataframe of all units with their statistics.

    '''
    for unit_name in db:
        for item_index in range(len(db[unit_name]['scores'])):
            db[unit_name][f'I{item_index+1}'] = db[unit_name]["scores"][item_index]
    df = pd.DataFrame(db).T

    df["unit_code"] = df["unit_code"].apply(lambda x: x[1:])
    df['agg_score'] = df['scores'].apply(
        lambda entry: [sum(item[measure] for item in entry)/len(entry) for measure in range(2)]
    ) # Determine mean of all items over mean and median
    df = df.drop(["scores", ], axis=1)
    df["school"] = df["unit_code"].str[0:3]
    df["Response Rate"] = df['Responses']/df['Invited']*100

    df = df.reindex(columns=["code", "unit_code", "school", "Level", "Season"]+[
                    f'I{i}' for i in range(1, 14)]+["agg_score", "Invited", "Responses", "Response Rate"])
    return df

In [40]:
to_dataframe(thing)

,code,unit_code,school,Level,Season,I1,I2,I3,I4,I5,...,I8,I9,I10,I11,I12,I13,agg_score,Invited,Responses,Response Rate
ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01,ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01,ACB1020,ACB,1,2020_S1,"[4.12, 4.18]","[4.34, 4.33]","[3.98, 4.0]","[3.83, 3.93]","[3.85, 3.92]",...,"[3.8, 3.97]","[4.03, 4.03]","[4.05, 4.05]","[4.0, 4.09]","[4.22, 4.25]","[3.85, 3.89]","[4.001538461538462, 4.053076923076923]",115,41,35.652174
ACB1120_PENINSULA_ON-CAMPUS_ON_S1-01,ACB1120_PENINSULA_ON-CAMPUS_ON_S1-01,ACB1120,ACB,1,2020_S1,"[3.92, 4.0]","[4.08, 4.08]","[3.92, 3.94]","[3.77, 3.75]","[3.92, 4.0]",...,"[3.77, 3.86]","[4.08, 4.06]","[4.0, 4.0]","[4.08, 4.06]","[3.92, 3.94]","[4.0, 4.0]","[3.946923076923077, 3.9676923076923076]",65,13,20.0
ACB2120_PENINSULA_ON-CAMPUS_ON_S1-01,ACB2120_PENINSULA_ON-CAMPUS_ON_S1-01,ACB2120,ACB,2,2020_S1,"[4.0, 4.63]","[4.29, 4.63]","[3.86, 4.25]","[4.0, 4.63]","[3.86, 4.0]",...,"[3.57, 4.0]","[3.67, 4.5]","[4.14, 4.63]","[4.0, 4.63]","[3.86, 4.63]","[3.86, 4.63]","[3.9315384615384614, 4.470769230769232]",22,7,31.818182
ACB2220_PENINSULA_ON-CAMPUS_ON_S1-01,ACB2220_PENINSULA_ON-CAMPUS_ON_S1-01,ACB2220,ACB,2,2020_S1,"[4.5, 4.5]","[4.33, 4.5]","[4.33, 4.5]","[4.17, 4.5]","[4.0, 4.5]",...,"[4.33, 4.5]","[4.25, 4.5]","[4.67, 4.75]","[4.2, 4.25]","[4.5, 4.5]","[4.33, 4.5]","[4.341538461538462, 4.5]",10,6,60.0
ACC1100_CLAYTON_ON-CAMPUS_ON_S1-01,ACC1100_CLAYTON_ON-CAMPUS_ON_S1-01,ACC1100,ACC,1,2020_S1,"[4.22, 4.27]","[4.1, 4.17]","[4.19, 4.26]","[3.87, 3.96]","[4.06, 4.12]",...,"[4.06, 4.14]","[4.19, 4.22]","[4.35, 4.39]","[4.13, 4.21]","[4.03, 4.16]","[3.89, 3.97]","[4.111538461538462, 4.185384615384615]",506,145,28.656126


In [26]:
thing[' ACB1020_PENINSULA_ON-CAMPUS_ON_S1-01 ']['scores'][0]

array([4.12, 4.18])